Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [56]:
NAME = ""
COLLABORATORS = ""

---

# An Overview of Pandas GroupBy

In [57]:
import numpy as np
import pandas as pd

This exercise is inspired by Wes McKinney's [Python for Data Analysis](http://proquest.safaribooksonline.com.libproxy.berkeley.edu/book/programming/python/9781491957653)

In [58]:
df = pd.read_csv("elections.csv")
df.head()

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss


## Grouping a Series by a Series

Let's group the `%` Series by the `Party` Series. A call to [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) does that, but what is the object that results?

In [59]:
percent_grouped_by_party = df['%'].groupby(df['Party'])
type(percent_grouped_by_party)

pandas.core.groupby.SeriesGroupBy

As we see, `percent_grouped_by_party` is **NOT** a familiar object like a DataFrame, Series, or Index. Instead, it's a `SeriesGroupBy` object. A `SeriesGroupBy` consists of `groups`, one for each of the distinct values of the `Party` column. If we ask to see these groups, we'll be able to see which indices in the original DataFrame correspond to each group.

In [60]:
percent_grouped_by_party.groups

{'Democratic': Int64Index([1, 4, 6, 7, 10, 13, 15, 17, 19, 21], dtype='int64'),
 'Independent': Int64Index([2, 9, 12], dtype='int64'),
 'Republican': Int64Index([0, 3, 5, 8, 11, 14, 16, 18, 20, 22], dtype='int64')}

The `percent_grouped_by_party` object is capable of making computations across all these groups. For example, if we call the `mean` method of the `SeriesGroupBy` class, we'll get a new `Series` containing the mean of the "Democratic" `Series`, the mean of the "Independent" `Series`, and the mean of the "Republican" `Series`.

In [61]:
percent_grouped_by_party.mean()

Party
Democratic     46.53
Independent    11.30
Republican     47.86
Name: %, dtype: float64

The output of the `mean` methood is a regular ole pandas Series.

In [62]:
type(percent_grouped_by_party.mean())

pandas.core.series.Series

`SeriesGroupBy` objects have many other handy methods, e.g. max and min.

In [8]:
percent_grouped_by_party.max()

Party
Democratic     52.9
Independent    18.9
Republican     58.8
Name: %, dtype: float64

In [9]:
percent_grouped_by_party.min()

Party
Democratic     37.6
Independent     6.6
Republican     37.4
Name: %, dtype: float64

In [10]:
percent_grouped_by_party.size()

Party
Democratic     10
Independent     3
Republican     10
Name: %, dtype: int64

In [11]:
percent_grouped_by_party.first()

Party
Democratic     41.0
Independent     6.6
Republican     50.7
Name: %, dtype: float64

We can iterate over a `SeriesGroupBy` object -- though we're doing this just for educational purposes and you'll probably never actually do this with a real `SeriesGroupBy`. As we iterate we get pairs of `(name, group)`, where `name` is a String label for the group, and `group` is a `Series` corresponding to the all the values from the given group.

In [12]:
from IPython.display import display  # like print, but for complex objects

for name, group in percent_grouped_by_party:
    print('Name:', name)
    print(type(group))
    display(group)

Name: Democratic
<class 'pandas.core.series.Series'>


1     41.0
4     37.6
6     45.6
7     43.0
10    49.2
13    48.4
15    48.3
17    52.9
19    51.1
21    48.2
Name: %, dtype: float64

Name: Independent
<class 'pandas.core.series.Series'>


2      6.6
9     18.9
12     8.4
Name: %, dtype: float64

Name: Republican
<class 'pandas.core.series.Series'>


0     50.7
3     58.8
5     53.4
8     37.4
11    40.7
14    47.9
16    50.7
18    45.7
20    47.2
22    46.1
Name: %, dtype: float64

In [15]:
df1 = pd.DataFrame([('A', 3, 'ak'), ['B', 1, 'tx'], ['C', 4, 'fl'], 
                  ['A', 1, 'hi'], ['B', 5, 'mi'], ['C', 9, 'ak'], 
                  ['A', 2, 'ca'], ['C', 5, 'sd'], ['B', 6, 'nc']],
                 columns = ['col1', 'col2', 'col3'])

In [16]:
df1

,col1,col2,col3
0,A,3,ak
1,B,1,tx
2,C,4,fl
3,A,1,hi
4,B,5,mi
5,C,9,ak
6,A,2,ca
7,C,5,sd
8,B,6,nc


In [18]:
df1_groupde_by_col1 = df1.groupby('col1')

In [22]:
df1_groupde_by_col1.mean()

,col2
col1,
A,2
B,4
C,6


## Grouping a Series by Multiple Series

We can also group a Series by multiple Series. For example, suppose we want to track all combinations of `{'Democratic', 'Republican', and 'Independent'}` and `{'win', 'loss'}`. 

In [29]:
percent_grouped_by_party_and_result = df['%'].groupby([df['Party'], df['Result']])
percent_grouped_by_party_and_result.groups

{('Democratic', 'loss'): Int64Index([1, 4, 6, 13, 15, 21], dtype='int64'),
 ('Democratic', 'win'): Int64Index([7, 10, 17, 19], dtype='int64'),
 ('Independent', 'loss'): Int64Index([2, 9, 12], dtype='int64'),
 ('Republican', 'loss'): Int64Index([8, 11, 18, 20], dtype='int64'),
 ('Republican', 'win'): Int64Index([0, 3, 5, 14, 16, 22], dtype='int64')}

Given this groupby object, we can compute the average percentage earned every time each of the parties won and lost the presidential election. We see that at least between 1980 and 2016, the Republicans have typically lost and won their elections by wider margins.

In [30]:
percent_grouped_by_party_and_result.mean()

Party        Result
Democratic   loss      44.850000
             win       49.050000
Independent  loss      11.300000
Republican   loss      42.750000
             win       51.266667
Name: %, dtype: float64

The careful reader will note that the returned object looks a little funny. It seems to have two indexes! If we check the type of this object, we'll see it's just a regular Series.

In [31]:
type(percent_grouped_by_party_and_result.mean())

pandas.core.series.Series

However if we request to see the index of this `Series`, we see that it is a "MultiIndex", which is a special type of index used for data that is indexed in two or more ways.

In [32]:
percent_grouped_by_party_and_result.mean().index

MultiIndex(levels=[['Democratic', 'Independent', 'Republican'], ['loss', 'win']],
           labels=[[0, 0, 1, 2, 2], [0, 1, 0, 0, 1]],
           names=['Party', 'Result'])

We won't go into MultiIndexes formally today, but be aware that it is possible to index into a Series that has a MultiIndex. The sytnax is about exactly as you'd expect, for example:

In [33]:
percent_grouped_by_party_and_result.mean()["Democratic"]

Result
loss    44.85
win     49.05
Name: %, dtype: float64

## Grouping a DataFrame by a Series

We can also group an entire dataframe by one or more Series. This results in a `DataFrameGroupBy` object as the result:

In [34]:
everything_grouped_by_party = df.groupby('Party')
everything_grouped_by_party

As in our previous example, this object contains three `group` objects, one for each party label.

In [35]:
everything_grouped_by_party.groups

{'Democratic': Int64Index([1, 4, 6, 7, 10, 13, 15, 17, 19, 21], dtype='int64'),
 'Independent': Int64Index([2, 9, 12], dtype='int64'),
 'Republican': Int64Index([0, 3, 5, 8, 11, 14, 16, 18, 20, 22], dtype='int64')}

Just as with `SeriesGroupBy` objects, we can iterate over a `DataFrameGroupBy` object to understand what is effectively inside.

In [36]:
for n, g in everything_grouped_by_party:
    print('name:', n)
    display(g)

name: Democratic


,Candidate,Party,%,Year,Result
1,Carter,Democratic,41.0,1980,loss
4,Mondale,Democratic,37.6,1984,loss
6,Dukakis,Democratic,45.6,1988,loss
7,Clinton,Democratic,43.0,1992,win
10,Clinton,Democratic,49.2,1996,win
13,Gore,Democratic,48.4,2000,loss
15,Kerry,Democratic,48.3,2004,loss
17,Obama,Democratic,52.9,2008,win
19,Obama,Democratic,51.1,2012,win
21,Clinton,Democratic,48.2,2016,loss


name: Independent


,Candidate,Party,%,Year,Result
2,Anderson,Independent,6.6,1980,loss
9,Perot,Independent,18.9,1992,loss
12,Perot,Independent,8.4,1996,loss


name: Republican


,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
3,Reagan,Republican,58.8,1984,win
5,Bush,Republican,53.4,1988,win
8,Bush,Republican,37.4,1992,loss
11,Dole,Republican,40.7,1996,loss
14,Bush,Republican,47.9,2000,win
16,Bush,Republican,50.7,2004,win
18,McCain,Republican,45.7,2008,loss
20,Romney,Republican,47.2,2012,loss
22,Trump,Republican,46.1,2016,win


And just like `SeriesGroupBy` objects, we can apply methods like `mean` to compute the mean of each group. Since a `DataFrameGroupBy` is linked to the entire original dataframe (instead of to a single column from the dataframe), we calculate a mean for every numerical column. In this example below, we get the mean vote earned (as before), and the mean year (which isn't a useful quantity).

In [37]:
everything_grouped_by_party.mean()

,%,Year
Party,,
Democratic,46.53,1998.000000
Independent,11.30,1989.333333
Republican,47.86,1998.000000


Where did all the other columns go in the mean above? They are *nuisance columns*, which get automatically eliminated from an operation where it doesn't make sense (such as a numerical mean).

## Grouping a DataFrame by Multiple Series

DataFrames may also be grouped by multiple series at once. For example, we can repeat what we did with a Series above and group the entire DataFrame by Party and Result. After aggregation, we end up with a DataFrame that has a MultiIndex.

In [ ]:
everything_grouped_by_party_and_result=df.groupby([df['Party'], df['Result']])

In [ ]:
everything_grouped_by_party_and_result.max()

The resulting DataFrame above is pretty strange. We'll observe that Walter Mondale did not run for office in 2016! Make sure you understand why this is happening, as this sort of thing is a common mistake made by people who don't fully understand how pandas works.

Challenge: Try to figure out how to generate a table like the one above, except that each entry should represent all attributes of the candidate who got the maximum vote in that category. For example, the Republican Win category should feature Reagan in 1984 with 58.8% of the vote. The answer is a few cells below this one. There's a hint a few cells down.

Hint: Consider using `sort_values` and `first` somehow.

In [ ]:
df.sort_values("%", ascending=False).groupby([df['Party'], df['Result']]).first()

The result of an aggregation function applied to a DataFrameGroupBy 

## Custom Aggregation Functions

As described above, both `SeriesGroupBy` and `DataFrameGroupBy` objects have lots of handy methods for computing aggregate values for groups, e.g.

In [ ]:
percent_grouped_by_party.min()

In [ ]:
everything_grouped_by_party.median()

It turns out that all of these GroupBy methods are just shorthand for a more powerful and universal method of our GroupBy objects called agg. For example, `.min()` is just shorthand for `.agg(min)`, where `min` refers to the function `min`.

In [ ]:
percent_grouped_by_party.agg(min)

Naturally, we can define our own custom aggregation functions. For example, the function below returns the first item in a series.

In [ ]:
def average_of_first_and_last(series):
    return (series.iloc[0] + series.iloc[-1])/2

We can supply this function as a custom aggregation function for each series. As you can see, nuisance columns are automatically removed.

In [ ]:
everything_grouped_by_party.agg(average_of_first_and_last)

agg is fundamental to our use of GroupBy objects. Indeed, you will rarely call `groupby` without also calling `agg`, at least implicitly. We can summarize the grouping process with the following diagram, inspired by a similar diagram created by DS100 instructor Joey Gonzales. Diagram source at [this link](https://docs.google.com/presentation/d/1FrYg6yd6B-CIgfWLWm4W8vBhfmJ6Qt9dKkN-mlN5AKU/edit#slide=id.g4131093782_0_40).

![groupby](groupby_overview.png)

The result of calling `groupby` then `agg` on a `Series` is also a `Series`, and the result of calling `groupby` then `agg` on a `DataFrame` is also typically a `DataFrame`, though there are exceptions, e.g. if you use the aggregation function `size`, you will get back a `Series`.

In [ ]:
def list_of_first_and_last(series):
    return [series.iloc[0], series.iloc[-1]]

In [ ]:
everything_grouped_by_party.agg(list_of_first_and_last)

## Using Groups to Filter Datasets

In [38]:
everything_grouped_by_party = df.groupby('Party')

In [39]:
everything_grouped_by_party.filter(lambda subframe: subframe["Year"].max() < 2000)

,Candidate,Party,%,Year,Result
2,Anderson,Independent,6.6,1980,loss
9,Perot,Independent,18.9,1992,loss
12,Perot,Independent,8.4,1996,loss


In [40]:
everything_grouped_by_year = df.groupby('Year')

In [41]:
everything_grouped_by_year.filter(lambda subframe: subframe["%"].sum() < 97)

,Candidate,Party,%,Year,Result
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss
13,Gore,Democratic,48.4,2000,loss
14,Bush,Republican,47.9,2000,win
21,Clinton,Democratic,48.2,2016,loss
22,Trump,Republican,46.1,2016,win


A visual picture of how filtering works is shown below.

![filter](filter_overview.png)

## Using isin for filtering

This isn't actually related to `groupby` at all, but it seemed natural to bring it up here. 

Sometimes we want to restrict our attention to only rows where certain values appear. For example, we saw last time how we'd look at only rows that contain "Democratic" candidates.

In [42]:
df[df["Party"] == "Democratic"]

,Candidate,Party,%,Year,Result
1,Carter,Democratic,41.0,1980,loss
4,Mondale,Democratic,37.6,1984,loss
6,Dukakis,Democratic,45.6,1988,loss
7,Clinton,Democratic,43.0,1992,win
10,Clinton,Democratic,49.2,1996,win
13,Gore,Democratic,48.4,2000,loss
15,Kerry,Democratic,48.3,2004,loss
17,Obama,Democratic,52.9,2008,win
19,Obama,Democratic,51.1,2012,win
21,Clinton,Democratic,48.2,2016,loss


Suppose we wanted to filter such that all Republicans and Democrats appeared. One ugly way to do this would be `df[(df["Party"] == "Democratic") | (df["Party"] == "Republican")]`. However, a better idea is to use the `isin` method.

In [43]:
df[(df["Party"] == "Democratic") | (df["Party"] == "Republican")]

df[df["Party"].isin(["Republican", "Democratic"])]

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss
5,Bush,Republican,53.4,1988,win
6,Dukakis,Democratic,45.6,1988,loss
7,Clinton,Democratic,43.0,1992,win
8,Bush,Republican,37.4,1992,loss
10,Clinton,Democratic,49.2,1996,win
11,Dole,Republican,40.7,1996,loss


## Grouping over a different dimension (bonus topic, less often useful)

Above, we've been grouping data along the rows, using column keys as our selectors.  But we can also group along the columns, for example we can group by how many times the letter a appears in the column name.

In [44]:
grouped = df.groupby(lambda x: x.count('a'), axis=1)
for dtype, group in grouped:
    print(dtype)
    display(group)

0


,%,Result
0,50.7,win
1,41.0,loss
2,6.6,loss
3,58.8,win
4,37.6,loss
5,53.4,win
6,45.6,loss
7,43.0,win
8,37.4,loss
9,18.9,loss


1


,Party,Year
0,Republican,1980
1,Democratic,1980
2,Independent,1980
3,Republican,1984
4,Democratic,1984
5,Republican,1988
6,Democratic,1988
7,Democratic,1992
8,Republican,1992
9,Independent,1992


2


,Candidate
0,Reagan
1,Carter
2,Anderson
3,Reagan
4,Mondale
5,Bush
6,Dukakis
7,Clinton
8,Bush
9,Perot


In [45]:
grouped.max().head(5)

,0,1,2
0,50.7,1980,Reagan
1,41.0,1980,Carter
2,6.6,1980,Anderson
3,58.8,1984,Reagan
4,37.6,1984,Mondale


In [46]:
financial_data = pd.read_csv("financial_data.csv", index_col = 0)

In [47]:
financial_data.head(5)

,1/3/1984,2/1/1984,3/1/1984,4/2/1984,5/1/1984,6/1/1984,7/2/1984,8/1/1984,9/4/1984,10/1/1984,...,3/1/2016,4/1/2016,5/1/2016,6/1/2016,7/1/2016,8/1/2016,9/1/2016,10/1/2016,11/1/2016,12/1/2016
Date,,,,,,,,,,,,,,,,,,,,,
2 Year Treasury Yield,10.64,10.79,11.31,11.69,12.47,12.91,12.88,12.43,12.20,11.60,...,0.88,0.77,0.82,0.73,0.67,0.74,0.77,0.84,0.98,1.20
30 Year Mortgage Rate,13.37,13.23,13.39,13.65,13.94,14.42,14.67,14.47,14.35,14.13,...,3.69,3.61,3.60,3.57,3.44,3.44,3.46,3.47,3.77,4.20
Federal Funds Rate,9.56,9.59,9.91,10.29,10.32,11.06,11.23,11.64,11.30,9.99,...,0.36,0.37,0.37,0.38,0.39,0.40,0.40,0.40,0.41,0.54


In [48]:
def get_year(datestr):
    return datestr.split('/')[2]

In [49]:
grouped_by_year = financial_data.groupby(get_year, axis=1)
grouped_by_year.mean()

,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Date,,,,,,,,,,,,,,,,,,,,,
2 Year Treasury Yield,11.645833,9.271667,6.870833,7.406667,8.098333,8.575833,8.157500,6.4850,4.770833,4.048333,...,4.357500,2.004167,0.956667,0.702500,0.451667,0.276667,0.307500,0.462500,0.685833,0.835833
30 Year Mortgage Rate,13.870000,12.424167,10.181667,10.195000,10.334167,10.324167,10.129167,9.2500,8.400000,7.325833,...,6.342500,6.038333,5.041667,4.691667,4.457500,3.656667,3.983333,4.172500,3.849167,3.648333
Federal Funds Rate,10.225000,8.100833,6.805000,6.657500,7.568333,9.216667,8.099167,5.6875,3.521667,3.022500,...,5.019167,1.927500,0.160000,0.175000,0.101667,0.140000,0.107500,0.089167,0.132500,0.395000


## Example from Lecture Figures

For your convenience, we've provided the dataframe from lecture for the groupby examples.

In [50]:
lex = pd.DataFrame({"name": ["A", "B", "C", "A", "B", "C", "A", "D", "B"],
             "num": [3, 1, 4, 1, 5, 9, 2, 5, 6]})

In [51]:
lex.groupby("name").filter(lambda sf: sf["num"].sum() > 10)

,name,num
1,B,1
2,C,4
4,B,5
5,C,9
8,B,6


## Pivot Tables

Recall from before that we were able to group the % Series by the "Party" and "Result" Series, allowing us to understand the average vote earned by each party under each election result.

In [52]:
percent_grouped_by_party_and_result = df['%'].groupby([df['Party'], df['Result']])
percent_grouped_by_party_and_result.mean()

Party        Result
Democratic   loss      44.850000
             win       49.050000
Independent  loss      11.300000
Republican   loss      42.750000
             win       51.266667
Name: %, dtype: float64

Because we called `groupby` on a Series, the result of our aggregation operation was also a Series. However, I believe this data is more naturally expressed in a tabular format, with Party as the rows, and Result as the columns. The `pivot_table` operation is the natural way to achieve this data format.

In [53]:
df_pivot = df.pivot_table(
    index='Party', # the rows (turned into index)
    columns='Result', # the column values
    values='%', # the field(s) to processed in each group
    aggfunc=np.mean, # group operation
)
df_pivot.head()

Result,loss,win
Party,,
Democratic,44.85,49.050000
Independent,11.30,NaN
Republican,42.75,51.266667


In [55]:
df_pivot1 = df.pivot_table(
    index='Party', # the rows (turned into index)
    columns='Result', # the column values
    values='%', # the field(s) to processed in each group
    aggfunc=np.min, # group operation
)
df_pivot1.head()

Result,loss,win
Party,,
Democratic,37.6,43.0
Independent,6.6,NaN
Republican,37.4,46.1


The basic idea is that you specify a Series to be the `index` (i.e. rows) and a Series to be the `columns`. The data in the specified `values` is then grouped by all possible combinations of values that occur in the `index` and `columns` Series. These groups are then aggregated using the `aggfunc`, and arranged into a table that matches the requested `index` and `columns`. The diagram below summarizes how pivot tables are formed. (Diagram inspired by Joey Gonzales). Diagram source at [this link](https://docs.google.com/presentation/d/1FrYg6yd6B-CIgfWLWm4W8vBhfmJ6Qt9dKkN-mlN5AKU/edit#slide=id.g4131093782_0_89).

![groupby](pivot_table_overview.png)

For more on pivot tables, see [this excellent tutorial](http://pbpython.com/pandas-pivot-table-explained.html) by Chris Moffitt.

## List Arguments to pivot_table (Extra)

The arguments to our pivot_table method can also be lists. A few examples are given below.

If we pivot such that only our `columns` argument is a list, we end up with columns that are MultiIndexed.

In [ ]:
df_pivot = df.pivot_table(
    index='Party', # the rows (turned into index)
    columns=['Result', 'Candidate'], # the column values
    values='%', # the field(s) to processed in each group
    aggfunc=np.mean, # group operation
)
df_pivot.head()

If we pivot such that only our `index` argument is a list, we end up with rows that are MultiIndexed.

In [ ]:
df_pivot = df.pivot_table(
    index=['Party', 'Candidate'], # the rows (turned into index)
    columns='Result',# the column values
    values='%', # the field(s) to processed in each group
    aggfunc=np.mean, # group operation
)
df_pivot

If we pivot such that only our values argument is a list, then we again get a DataFrame with multi-indexed Columns.

In [ ]:
df_pivot = df.pivot_table(
    index='Party', # the rows (turned into index)
    columns='Result',# the column values
    values=['%', 'Year'], # the field(s) to processed in each group
    aggfunc=np.mean, # group operation
)
df_pivot

Feel free to experiment with other possibilities!

## Submission

You're done!

Before submitting this assignment, ensure to:

1. Restart the Kernel (in the menubar, select Kernel->Restart & Run All)
2. Validate the notebook by clicking the "Validate" button

Finally, make sure to **submit** the assignment via the Assignments tab in Datahub